# 1. Start Spark Session and Import Libraries

In [1]:
# import libraries
import findspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, DateType, BooleanType
from pyspark.ml import PipelineModel
from pyspark.ml.classification import RandomForestClassificationModel


findspark.init()

# load kafka and mongo packages
# this was our attempt at getting mongo working however we ended up using parquet
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1"
mongo_package = "org.mongodb.spark:mongo-spark-connector_2.13:10.2.1"

spark = SparkSession.builder \
    .appName("FlightDelayStreamingPrediction") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.jars.packages", f"{kafka_package},{mongo_package}") \
    .getOrCreate()



:: loading settings :: url = jar:file:/home/hadoop/spark-3.5.4/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jj/.ivy2/cache
The jars for the packages stored in: /home/jj/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-459996d1-6106-4bd5-b57f-2ca22ddded26;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13

# 2. Load Models & Pipeline

In [2]:
# using our GBT model
from pyspark.ml.classification import GBTClassificationModel

pipeline_path = "./flight_delay_gbt_pipeline_model"
model_path = "./flight_delay_gbt_model"

loaded_pipeline_model = PipelineModel.load(pipeline_path)
loaded_gbt_model = GBTClassificationModel.load(model_path)


# 3. Create Kafka Topic

In [3]:
kafka_topic = "flight_data_stream"
kafka_bootstrap_servers = "localhost:9092"

kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "latest") \
    .load()

kafka_df = kafka_df.selectExpr("CAST(value AS STRING)")
kafka_df.printSchema()

root
 |-- value: string (nullable = true)



# 4. Prepare for Streaming

In [4]:
json_schema = StructType([
    StructField("FL_DATE", StringType(), True),
    StructField("AIRLINE", StringType(), True),
    StructField("AIRLINE_CODE", StringType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("CRS_DEP_TIME", IntegerType(), True),
    StructField("CRS_ARR_TIME", IntegerType(), True),
    StructField("CRS_ELAPSED_TIME", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
])

parsed_stream_df = kafka_df \
    .select(F.from_json(F.col("value"), json_schema).alias("data")) \
    .select("data.*")

parsed_stream_df = parsed_stream_df.withColumn("FL_DATE", F.to_date(F.col("FL_DATE"), "yyyy-MM-dd"))
parsed_stream_df = parsed_stream_df.withColumn("DEP_HOUR", (F.col("CRS_DEP_TIME") / 100).cast("integer"))
parsed_stream_df = parsed_stream_df.withColumn("DEP_MINUTE", (F.col("CRS_DEP_TIME") % 100).cast("integer"))
parsed_stream_df = parsed_stream_df.withColumn("ARR_HOUR", (F.col("CRS_ARR_TIME") / 100).cast("integer"))
parsed_stream_df = parsed_stream_df.withColumn("ARR_MINUTE", (F.col("CRS_ARR_TIME") % 100).cast("integer"))
parsed_stream_df = parsed_stream_df.withColumn("DEP_DAY_OF_WEEK", F.dayofweek(F.col("FL_DATE")))
parsed_stream_df = parsed_stream_df.withColumn("DEP_MONTH", F.month(F.col("FL_DATE")))
parsed_stream_df = parsed_stream_df.withColumn("DEP_DAY_OF_MONTH", F.dayofmonth(F.col("FL_DATE")))
parsed_stream_df = parsed_stream_df.withColumn("DEP_WEEK_OF_YEAR", F.weekofyear(F.col("FL_DATE")))
parsed_stream_df = parsed_stream_df.withColumn("IS_WEEKEND", F.when(F.col("DEP_DAY_OF_WEEK").isin([1, 7]), 1).otherwise(0))
parsed_stream_df = parsed_stream_df.withColumn("DISTANCE_PER_MINUTE", F.col("DISTANCE") / (F.col("CRS_ELAPSED_TIME") + 1e-6))


feature_columns = [
    "AIRLINE_CODE", "ORIGIN", "DEST", "CRS_ELAPSED_TIME", "DISTANCE",
    "DEP_HOUR", "DEP_MINUTE", "ARR_HOUR", "ARR_MINUTE",
    "DEP_DAY_OF_WEEK", "DEP_MONTH", "DEP_DAY_OF_MONTH", "DEP_WEEK_OF_YEAR",
    "IS_WEEKEND", "DISTANCE_PER_MINUTE"
]
parsed_stream_df = parsed_stream_df.dropna(subset=feature_columns + ["FL_DATE"])
parsed_stream_df.printSchema()

processed_stream_df = loaded_pipeline_model.transform(parsed_stream_df)

predictions_df = loaded_gbt_model.transform(processed_stream_df)

output_df = predictions_df.select(
    "FL_DATE",
    "AIRLINE_CODE",
    "ORIGIN",
    "DEST",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "DISTANCE",
    "prediction",
    "probability"
)

output_df = output_df.withColumn(
    "Prediction_Label",
     F.when(F.col("prediction") == 1, "Severe Delay Predicted")
     .otherwise("No Severe Delay Predicted")
)

root
 |-- FL_DATE: date (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- DEP_HOUR: integer (nullable = true)
 |-- DEP_MINUTE: integer (nullable = true)
 |-- ARR_HOUR: integer (nullable = true)
 |-- ARR_MINUTE: integer (nullable = true)
 |-- DEP_DAY_OF_WEEK: integer (nullable = true)
 |-- DEP_MONTH: integer (nullable = true)
 |-- DEP_DAY_OF_MONTH: integer (nullable = true)
 |-- DEP_WEEK_OF_YEAR: integer (nullable = true)
 |-- IS_WEEKEND: integer (nullable = false)
 |-- DISTANCE_PER_MINUTE: double (nullable = true)



In [5]:
# after you run this, it will continue running while streaming the data to the website
# make sure kafka producer and flask app are running in two terminals

prob_udf = F.udf(lambda prob: float(prob[1]), DoubleType())
output_df = output_df.withColumn("Probability_Severe_Delay", prob_udf(F.col("probability")))

output_df.printSchema()

mongo_uri = "mongodb://127.0.0.1/flightdb.flight_predictions"

mongo_query = output_df \
    .select("FL_DATE", "AIRLINE_CODE", "ORIGIN", "DEST", "CRS_DEP_TIME", "Prediction_Label", "Probability_Severe_Delay") \
    .writeStream \
    .format("mongodb") \
    .option("spark.mongodb.connection.uri", mongo_uri) \
    .option("checkpointLocation", "./mongodb_checkpoint") \
    .outputMode("append") \
    .start()


# Console output
console_query = output_df \
    .select("FL_DATE", "AIRLINE_CODE", "ORIGIN", "DEST", "CRS_DEP_TIME", "Prediction_Label", "Probability_Severe_Delay") \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

# File output
file_output_path = "./streaming_predictions_output"
file_query = output_df \
    .select("FL_DATE", "AIRLINE_CODE", "ORIGIN", "DEST", "CRS_DEP_TIME", "Prediction_Label", "Probability_Severe_Delay") \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", file_output_path) \
    .option("checkpointLocation", "./streaming_predictions_checkpoint") \
    .start()

# Wait for all queries
try:
    while True:
        if any([q.isActive for q in [console_query, file_query, mongo_query]]):
            pass
        else:
            break
except KeyboardInterrupt:
    for q in [console_query, file_query, mongo_query]:
        q.stop()
    print("Streaming query interrupted by user.")


root
 |-- FL_DATE: date (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- prediction: double (nullable = false)
 |-- probability: vector (nullable = true)
 |-- Prediction_Label: string (nullable = false)
 |-- Probability_Severe_Delay: double (nullable = true)



25/04/29 19:22:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/29 19:22:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-32f310d3-c7df-4425-9e12-5ebdbc4eb1b7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/29 19:22:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/29 19:22:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/29 19:22:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are 

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+------------+------+----+------------+-------------------------+------------------------+
|FL_DATE   |AIRLINE_CODE|ORIGIN|DEST|CRS_DEP_TIME|Prediction_Label         |Probability_Severe_Delay|
+----------+------------+------+----+------------+-------------------------+------------------------+
|2021-06-18|YV          |LGA   |IAH |1545        |No Severe Delay Predicted|0.48014302440868317     |
|2021-08-11|MQ          |CHS   |DFW |1143        |No Severe Delay Predicted|0.21053222968965968     |
|2019-06-29|OO          |BUR   |PDX |700         |No Severe Delay Predicted|0.19908754508180793     |
|2021-12-31|UA          |EWR   |DCA |1500        |No Severe Delay Predicted|0.43401713350705473     |
+----------+------------+------+----+------------+-------------------------+------------------------+



25/04/29 19:22:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:11 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

-------------------------------------------
Batch: 1
-------------------------------------------
+----------+------------+------+----+------------+-------------------------+------------------------+
|FL_DATE   |AIRLINE_CODE|ORIGIN|DEST|CRS_DEP_TIME|Prediction_Label         |Probability_Severe_Delay|
+----------+------------+------+----+------------+-------------------------+------------------------+
|2022-05-06|WN          |BWI   |SDF |810         |No Severe Delay Predicted|0.11363343460469144     |
|2023-01-31|UA          |ORD   |MSY |1955        |No Severe Delay Predicted|0.319110003438944       |
|2020-02-03|DL          |ATL   |PBI |825         |No Severe Delay Predicted|0.11887424379964595     |
|2019-03-29|UA          |SFO   |LAS |600         |No Severe Delay Predicted|0.14446658265712908     |
|2019-10-17|YX          |MSY   |DCA |541         |No Severe Delay Predicted|0.11641899870450456     |
|2020-07-18|OO          |ASE   |SFO |1430        |No Severe Delay Predicted|0.466205291

...

25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

-------------------------------------------
Batch: 76
-------------------------------------------
+----------+------------+------+----+------------+-------------------------+------------------------+
|FL_DATE   |AIRLINE_CODE|ORIGIN|DEST|CRS_DEP_TIME|Prediction_Label         |Probability_Severe_Delay|
+----------+------------+------+----+------------+-------------------------+------------------------+
|2020-08-16|AA          |IAH   |DFW |630         |No Severe Delay Predicted|0.1805218346213996      |
|2020-03-20|B6          |BOS   |PBI |1344        |No Severe Delay Predicted|0.293456008286616       |
|2020-11-22|B6          |FLL   |ATL |2145        |No Severe Delay Predicted|0.37134061346314584     |
|2020-12-09|DL          |SLC   |LAX |600         |No Severe Delay Predicted|0.12328799193944862     |
|2020-03-05|AA          |DFW   |MKE |1439        |No Severe Delay Predicted|0.27713687399098064     |
|2021-02-22|DL          |SAN   |SLC |615         |No Severe Delay Predicted|0.12822490

25/04/29 19:22:36 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/jj/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/jj/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Streaming query interrupted by user.


25/04/29 19:22:36 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 77, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/04/29 19:22:36 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 77, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/04/29 19:22:36 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:267)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:594)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:576)
	at scala.collection.Iterator$$anon$9.hasNext(Iterator.scala:576)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org